<a href="https://colab.research.google.com/github/Rishabh8827/Machine-Learning---Summer-Training/blob/main/Day%2010%20/LSTM_vs_bidirectional_lstm_neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Application of LSTM and Bidirectional Recurrent Neural Networks in Fake NEWS detection

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# importing necessary libraries 
import pandas as pd
import tensorflow as tf
import os
import re
import numpy as np
from string import punctuation
from zipfile import ZipFile
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [ ]:
# importing neural network libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, Bidirectional, LSTM, RNN, SpatialDropout1D, Bidirectional

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Datasets/fake-news/train.csv')
test = pd.read_csv('/content/drive/MyDrive/Datasets/fake-news/test.csv')
train_data = train.copy()
test_data = test.copy()

In [ ]:
train_data = train_data.set_index('id', drop = True)

In [ ]:
print(train_data.shape)
train_data.head()

(20800, 4)


,title,author,text,label
id,,,,
0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
print(test_data.shape)
test_data.head()

(5200, 4)


,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...


In [ ]:
# checking for missing values
train_data.isnull().sum()

title      558
author    1957
text        39
label        0
dtype: int64

out of 20,000 training samples, around 40 samples (bothering only the text column) have missing values. so we can drop them at once

In [ ]:
# dropping missing values from text columns alone. 
train_data[['title', 'author']] = train_data[['title', 'author']].fillna(value = 'Missing')
train_data = train_data.dropna()
train_data.isnull().sum()

title     0
author    0
text      0
label     0
dtype: int64

In [ ]:
length = []
[length.append(len(str(text))) for text in train_data['text']]
train_data['length'] = length
train_data.head()

,title,author,text,label,length
id,,,,,
0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,4930
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,4160
2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,7692
3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,3237
4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,938


In [ ]:
min(train_data['length']), max(train_data['length']), round(sum(train_data['length'])/len(train_data['length']))

(1, 142961, 4553)

we can keep 4500 as max features for training the neural network.

**minimum length is 1 ?? Looks like there are some outliers.**

In [ ]:
len(train_data[train_data['length'] < 50])

207

**There are 107 outliers in this dataset. Outliers can be removed. It is a good practice to check the outliers before removing them**

In [ ]:
train_data['text'][train_data['length'] < 50]

id
82                                                   
169                                                  
173                                   Guest   Guest  
196            They got the heater turned up on high.
295                                                  
                             ...                     
20350                         I hope nobody got hurt!
20418                                 Guest   Guest  
20431    \nOctober 28, 2016 The Mothers by stclair by
20513                                                
20636                              Trump all the way!
Name: text, Length: 207, dtype: object

*Mostly empty texts. They can be removed since they will surely guide the neural network in the wrong way*

In [ ]:
# dropping the outliers
train_data = train_data.drop(train_data['text'][train_data['length'] < 50].index, axis = 0)

In [ ]:
min(train_data['length']), max(train_data['length']), round(sum(train_data['length'])/len(train_data['length']))

(50, 142961, 4598)

In [47]:
max_features = 50

Preprocessing the Text before feeding it into the neural networks

In [48]:
# Tokenizing the text - converting the words, letters into counts or numbers. 
# We dont need to explicitly remove the punctuations. we have an inbuilt option in Tokenizer for this purpose
tokenizer = Tokenizer(num_words = max_features, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower = True, split = ' ')
tokenizer.fit_on_texts(texts = train_data['text'])
X = tokenizer.texts_to_sequences(texts = train_data['text'])

In [49]:
# now applying padding to make them even shaped.
X = pad_sequences(sequences = X, maxlen = max_features, padding = 'pre')

In [50]:
print(X.shape)
y = train_data['label'].values
print(y.shape)

(20554, 50)
(20554,)


In [51]:
# splitting the data training data for training and validation.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 101)

We got our training data preprocessed and ready for training the neural network. 

We have to create a neural network now

In [52]:
# LSTM Neural Network
lstm_model = Sequential(name = 'lstm_nn_model')
lstm_model.add(layer = Embedding(input_dim = max_features, output_dim = 120, name = '1st_layer'))
lstm_model.add(layer = LSTM(units = 120, dropout = 0.2, recurrent_dropout = 0.2, name = '2nd_layer'))
lstm_model.add(layer = Dropout(rate = 0.5, name = '3rd_layer'))
lstm_model.add(layer = Dense(units = 120,  activation = 'relu', name = '4th_layer'))
lstm_model.add(layer = Dropout(rate = 0.5, name = '5th_layer'))
lstm_model.add(layer = Dense(units = len(set(y)),  activation = 'sigmoid', name = 'output_layer'))
# compiling the model
lstm_model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [53]:
lstm_model_fit = lstm_model.fit(X_train, y_train, epochs = 1,batch_size=128)

129/129 [==============================] - 46s 287ms/step - loss: 0.4796 - accuracy: 0.7729


Constructing Bidirectional Neural Network

In [93]:
# Bidirectional neural Network
bi_lstm_model = Sequential(name = 'Bidirectional_nn_model')
bi_lstm_model.add(layer = Embedding(input_dim = max_features, output_dim = 120, name = '1st_layer'))
bi_lstm_model.add(layer = Bidirectional(LSTM(units = 120, dropout = 0.2, 
                          recurrent_dropout = 0.2, recurrent_activation = 'relu', 
                          activation = 'relu', name = '2nd_layer')))
bi_lstm_model.add(layer = Dropout(rate = 0.4, name = '3rd_layer'))
bi_lstm_model.add(layer = Dense(units = 120, activation = 'relu', name = '4th_layer'))
bi_lstm_model.add(layer = Dropout(rate = 0.2, name = '5th_layer'))
bi_lstm_model.add(layer = Dense(units = len(set(y)), activation = 'softmax', name = 'output_layer'))
# compiling the model
bi_lstm_model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [94]:
bi_lstm_model.summary()

Model: "Bidirectional_nn_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 1st_layer (Embedding)       (None, None, 120)         6000      
                                                                 
 bidirectional_2 (Bidirectio  (None, 240)              231360    
 nal)                                                            
                                                                 
 3rd_layer (Dropout)         (None, 240)               0         
                                                                 
 4th_layer (Dense)           (None, 120)               28920     
                                                                 
 5th_layer (Dropout)         (None, 120)               0         
                                                                 
 output_layer (Dense)        (None, 2)                 242       
                                            

In [95]:
bi_lstm_model_fit = bi_lstm_model.fit(X_train, y_train, epochs = 1,batch_size=128)

129/129 [==============================] - 80s 574ms/step - loss: nan - accuracy: 0.5116


Now preparing the test dataset

In [26]:
print(test.shape)
test_data = test.copy()
print(test_data.shape)

(5200, 4)
(5200, 4)


In [27]:
test_data = test_data.set_index('id', drop = True)
test_data.shape

(5200, 3)

**Filling the Missing values**

In [28]:
test_data = test_data.fillna(' ')
print(test_data.shape)
test_data.isnull().sum()

(5200, 3)


title     0
author    0
text      0
dtype: int64

In [57]:
tokenizer.fit_on_texts(texts = test_data['text'])
test_text = tokenizer.texts_to_sequences(texts = test_data['text'])

In [58]:
test_text = pad_sequences(sequences = test_text, maxlen = max_features, padding = 'pre')

Prediction:

In [97]:
lstm_prediction = lstm_model.predict(X_test)
lstm_prediction = np.argmax(lstm_prediction,axis=1)
lstm_prediction

array([1, 1, 1, ..., 0, 0, 1])

In [98]:
bi_lstm_prediction = bi_lstm_model.predict(X_test)
bi_lstm_prediction = np.argmax(bi_lstm_prediction,axis=1)
bi_lstm_prediction

array([0, 0, 0, ..., 0, 0, 0])

In [102]:
from sklearn.metrics import accuracy_score
scr1 = accuracy_score(y_test,lstm_prediction)
scr2 = accuracy_score(y_test,bi_lstm_prediction)

The LSTM predictions have more accuracy.